# Olist Silver Data Cleaning & Transformation

This notebook provides a business-focused overview of the Olist e-commerce platform’s silver data layer. The silver layer contains refined, quality-assured data transformed from Olist’s marketplace operations—including customers, orders, products, payments, reviews, sellers, and geolocation information.

## Business Context
Olist is a leading Brazilian e-commerce marketplace. The silver layer is essential for:
* Delivering reliable, actionable insights to marketing, operations, and customer service teams
* Supporting strategic decisions with trustworthy, up-to-date data
* Enabling advanced analytics, reporting, and machine learning initiatives

## Notebook Purpose
This notebook demonstrates how Olist’s raw data is cleaned, standardized, and validated to ensure accuracy and consistency. It helps:
* Identify and resolve data quality issues
* Prepare data for deeper business analysis and reporting
* Build confidence in the data used for decision-making

## Workflow Summary
* Data Cleaning: Address missing values, standardize formats, and impute key metrics for robust analysis
* Schema Inspection: Review the structure of transformed tables to understand available fields
* Data Validation: Confirm completeness and integrity of the silver layer

By following this workflow, business leaders, analysts, and teams can trust the data foundation for their insights, accelerate value creation, and drive growth across Olist’s marketplace.

In [0]:
customers_df = spark.read.table('olist_ecommerce.bronze.brz_customers')
orders_df = spark.read.table('olist_ecommerce.bronze.brz_orders')
order_items_df = spark.read.table('olist_ecommerce.bronze.brz_order_items')
products_df = spark.read.table('olist_ecommerce.bronze.brz_products')
payments_df = spark.read.table('olist_ecommerce.bronze.brz_order_payments')
reviews_df = spark.read.table('olist_ecommerce.bronze.brz_order_reviews')
sellers_df = spark.read.table('olist_ecommerce.bronze.brz_sellers')
geolocation_df = spark.read.table('olist_ecommerce.bronze.brz_geolocation')
product_category_name_translation_df = spark.read.table('olist_ecommerce.bronze.brz_product_category_name_translation')

In [0]:
customers_df.show()

# Identify Missing Values

In [0]:
from pyspark.sql.functions import *

catalog_name = 'olist_ecommerce'

In [0]:
# This function checks for null values in each column of the given DataFrame.
# Identifying missing data helps businesses ensure data quality, enabling more accurate analysis and decision-making.

def missing_values(df, df_name):
    print(f'Missing values in {df_name}:')
    df.select([count(when(col(c).isNull(), 1)).alias(c) for c in df.columns]).show()

In [0]:
# Check for missing (null) values in each column of the customers_df DataFrame.
missing_values(customers_df, 'customer')

In [0]:
# Check for missing (null) values in each column of the orders_df DataFrame.
missing_values(orders_df, 'orders')

In [0]:
# Check for missing (null) values in each column of the order_item_df DataFrame.
missing_values(order_items_df, 'order_item')

In [0]:
missing_values(payments_df,'payments')

# Handle Missing Values

1. Drop missing Values ( for non - critical columns )

2. Fill missing values ( for numerical columns )

3. Impute Missing Values ( for continous data )

In [0]:
# Drop rows with missing values in critical columns: 'order_id', 'customer_id', and 'order_status'
orders_df_silver =  orders_df.na.drop(subset=['order_id', 'customer_id', 'order_status'])

In [0]:
orders_df_silver.show()

In [0]:
# Fill missing values in 'order_delivered_customer_date' with a placeholder date '9999-12-31' to indicate undelivered orders.

from pyspark.sql.functions import when, col, lit, to_timestamp

orders_df_silver = orders_df_silver.withColumn(
    'order_delivered_customer_date',
    when(
        col('order_delivered_customer_date').isNull(),
        to_timestamp(lit('9999-12-31'))
    ).otherwise(col('order_delivered_customer_date'))
)

In [0]:
orders_df_silver.show()

# Impute missing values 

In [0]:
# Impute missing values in the 'payment_value' column using the mean strategy and create a new column 'payment_value_imputed'.
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=['payment_value'], outputCols=['payment_value_imputed']).setStrategy('mean')
payments_df_silver = imputer.fit(payments_df).transform(payments_df)

In [0]:
payments_df_silver.show()

# Standardizing the format


In [0]:
def print_schema(df,df_name):
    # Print the schema of the given DataFrame with its name
    print(f'schema of {df_name}:')
    df.printSchema()

In [0]:
print_schema(orders_df,'orders')

In [0]:
print_schema(customers_df,'customers')

In [0]:
print_schema(payments_df,'payments')

In [0]:
orders_df_silver.show()

In [0]:
# Convert 'order_purchase_timestamp' to date format to standardize the column for further analysis.
orders_df_silver = orders_df_silver.withColumn('order_purchase_timestamp', to_date(col('order_purchase_timestamp')))

In [0]:
orders_df_silver.show()

In [0]:
payments_df_silver.show()

In [0]:
# Standardize 'payment_type' values for consistency
payments_df_silver = payments_df_silver.withColumn('payment_type',when(col('payment_type')=='boleto','Bank Transfer')
                                                     .when(col('payment_type')=='credit_card','Credit Card')
                                                     .when(col('payment_type')=='debit_card','Debit Card')
                                                    .otherwise('other'))

In [0]:
payments_df_silver.show()

In [0]:
customers_df.printSchema()

In [0]:
customers_df.show()

In [0]:
customers_df_silver = customers_df.withColumn('customer_zip_code_prefix',col('customer_zip_code_prefix').cast('string'))

In [0]:
customers_df_silver.printSchema()

# Remove Duplicate Records

In [0]:
# Remove duplicate customer records based on 'customer_id'
customers_df_silver = customers_df_silver.dropDuplicates(['customer_id'])

# Data Transformation

In [0]:
# Join orders, order items, payments, and customers data to create a unified DataFrame with all order details.
order_with_details_silver = orders_df_silver.join(order_items_df,'order_id','left')\
.join(payments_df_silver,'order_id','left')\
.join(customers_df_silver,'customer_id','left')

In [0]:
display(order_with_details_silver.limit(20))

In [0]:
# Calculate the total payment value for each order
order_with_total_value_silver = order_with_details_silver.groupBy('order_id')\
.agg(sum('payment_value').alias('total_order_value'))

In [0]:
order_with_total_value_silver.show(5)

In [0]:
# Delivery Time Calculation from prev notebook(olist_bronze_exploration)

# Advance Transformation 

In [0]:
order_items_df.show()

In [0]:
# Calculate the 1st and 99th percentile price values to identify outlier thresholds
quantiles = order_items_df.approxQuantile('price',[0.01,0.99],0.0)
low_cutoff,high_cutoff = quantiles[0],quantiles[1]

In [0]:
 order_items_df.select('price').summary().show()

In [0]:
low_cutoff,high_cutoff

In [0]:
# Filter out order items with price values outside the 1st and 99th percentile thresholds (remove outliers)
order_items_df_silver = order_items_df.filter((col('price') >=low_cutoff) & (col('price') <=high_cutoff))

In [0]:
payments_df_silver.select('payment_installments').summary().show()

In [0]:
products_df.show()

In [0]:
# Categorize products based on their weight into 'Small', 'Medium', or 'Large'
products_df_silver = products_df.withColumn(
    'product_size_category',
    when(col('product_weight_g') <500,'Small')
    .when(col('product_weight_g').between(500,2000),'Medium')
    .otherwise('Large')
)

In [0]:
products_df_silver.show()

In [0]:
# Calculate total revenue per seller
seller_revenue_df = order_with_details_silver.groupBy('seller_id').agg(sum('payment_value').alias('total_revenue'))

display(seller_revenue_df)

In [0]:
# Save the unified order details DataFrame as a Delta table in the silver layer
order_with_details_silver.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("olist_ecommerce.silver.slv_order_with_details")

In [0]:
products_df_silver.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("olist_ecommerce.silver.slv_products")

In [0]:
spark.table("olist_ecommerce.silver.slv_order_with_details").printSchema()

In [0]:
spark.table("olist_ecommerce.silver.slv_products").printSchema()